En este archivo puedes escribir lo que estimes conveniente. Te recomendamos detallar tu solución y todas las suposiciones que estás considerando. Aquí puedes ejecutar las funciones que definiste en los otros archivos de la carpeta src, medir el tiempo, memoria, etc.

In [7]:
file_path = "../data/farmers-protest-tweets-2021-2-4.json"

In [56]:
# Importación de Librerías
import pandas as pd
import datetime
import re
from memory_profiler import memory_usage
import time
from typing import List, Tuple
from memory_profiler import memory_usage

In [57]:
# Función para medir tiempo y memoria
def measure_performance(func, *args):
    start_time = time.time()
    mem_usage = memory_usage((func, args), interval=0.1, timeout=None)
    end_time = time.time()
    exec_time = end_time - start_time
    max_mem_usage = max(mem_usage)
    return exec_time, max_mem_usage

In [9]:
# Cargar los Datos
data = pd.read_json(file_path, lines=True)
print(data.head())

                                                 url  \
0  https://twitter.com/ArjunSinghPanam/status/136...   
1  https://twitter.com/PrdeepNain/status/13645062...   
2  https://twitter.com/parmarmaninder/status/1364...   
3  https://twitter.com/anmoldhaliwal/status/13645...   
4  https://twitter.com/KotiaPreet/status/13645061...   

                       date  \
0 2021-02-24 09:23:35+00:00   
1 2021-02-24 09:23:32+00:00   
2 2021-02-24 09:23:22+00:00   
3 2021-02-24 09:23:16+00:00   
4 2021-02-24 09:23:10+00:00   

                                             content  \
0  The world progresses while the Indian police a...   
1  #FarmersProtest \n#ModiIgnoringFarmersDeaths \...   
2  ਪੈਟਰੋਲ ਦੀਆਂ ਕੀਮਤਾਂ ਨੂੰ ਮੱਦੇਨਜ਼ਰ ਰੱਖਦੇ ਹੋਏ \nਮੇ...   
3  @ReallySwara @rohini_sgh watch full video here...   
4  #KisanEktaMorcha #FarmersProtest #NoFarmersNoF...   

                                     renderedContent                   id  \
0  The world progresses while the Indian police a...  13645062

Ejecución de Funciones Python según la información solicitada

01_MEMORY -> Funcion enfocada en optimizar los recursos

In [30]:
# Función q1_memory
def q1_memory(file_path: str) -> List[Tuple[datetime.date, str]]:
    date_counts = {}
    user_counts = {}
    chunksize = 100000
    for chunk in pd.read_json(file_path, lines=True, chunksize=chunksize):
        chunk['date'] = pd.to_datetime(chunk['date'])
        chunk['date_only'] = chunk['date'].dt.date
        
        for date, group in chunk.groupby('date_only'):
            date_counts[date] = date_counts.get(date, 0) + len(group)
            user_counts[date] = user_counts.get(date, {})
            
            for user in group['user'].apply(lambda x: x['username']):
                user_counts[date][user] = user_counts[date].get(user, 0) + 1
    
    top_dates = sorted(date_counts.items(), key=lambda x: x[1], reverse=True)[:10]
    
    result = []
    for date, _ in top_dates:
        top_user = max(user_counts[date].items(), key=lambda x: x[1])[0]
        result.append((date, top_user))
    
    return result

In [31]:
# Probar la función q1_memory
result_q1_memory = q1_memory(file_path)
print(result_q1_memory)

[(datetime.date(2021, 2, 12), 'RanbirS00614606'), (datetime.date(2021, 2, 13), 'MaanDee08215437'), (datetime.date(2021, 2, 17), 'RaaJVinderkaur'), (datetime.date(2021, 2, 16), 'jot__b'), (datetime.date(2021, 2, 14), 'rebelpacifist'), (datetime.date(2021, 2, 18), 'neetuanjle_nitu'), (datetime.date(2021, 2, 15), 'jot__b'), (datetime.date(2021, 2, 20), 'MangalJ23056160'), (datetime.date(2021, 2, 23), 'Surrypuria'), (datetime.date(2021, 2, 19), 'Preetm91')]


Q1_TIME -> Funcion enfocada el optimizar el tiempo


In [28]:
## q1_time
def q1_time(file_path: str) -> List[Tuple[datetime.date, str]]:
    data = pd.read_json(file_path, lines=True)
    data['date'] = pd.to_datetime(data['date'])
    data['date_only'] = data['date'].dt.date
    tweet_counts = data['date_only'].value_counts().head(10)
    
    result = []
    for date in tweet_counts.index:
        daily_tweets = data[data['date_only'] == date]
        top_user = daily_tweets['user'].apply(lambda x: x['username']).value_counts().idxmax()
        result.append((date, top_user))
    
    return result

In [29]:
# Probar la función q1_time
result_q1_time = q1_time(file_path)
print(result_q1_time)

[(datetime.date(2021, 2, 12), 'RanbirS00614606'), (datetime.date(2021, 2, 13), 'MaanDee08215437'), (datetime.date(2021, 2, 17), 'RaaJVinderkaur'), (datetime.date(2021, 2, 16), 'jot__b'), (datetime.date(2021, 2, 14), 'rebelpacifist'), (datetime.date(2021, 2, 18), 'neetuanjle_nitu'), (datetime.date(2021, 2, 15), 'jot__b'), (datetime.date(2021, 2, 20), 'MangalJ23056160'), (datetime.date(2021, 2, 23), 'Surrypuria'), (datetime.date(2021, 2, 19), 'Preetm91')]


02_MEMORY -> Funcion enfocada en optimizar los recursos

In [35]:
# Función q2_memory
def q2_memory(file_path: str) -> List[Tuple[str, int]]:
    emoji_counts = {}
    emoji_pattern = re.compile("[\U00010000-\U0010FFFF]", flags=re.UNICODE)
    chunksize = 100000
    for chunk in pd.read_json(file_path, lines=True, chunksize=chunksize):
        chunk['emojis'] = chunk['content'].apply(lambda x: emoji_pattern.findall(x))
        for emojis in chunk['emojis']:
            for emoji in emojis:
                emoji_counts[emoji] = emoji_counts.get(emoji, 0) + 1
    top_emojis = sorted(emoji_counts.items(), key=lambda x: x[1], reverse=True)[:10]
    return top_emojis


In [36]:
# Probar la función q2_memory
result_q2_memory = q2_memory(file_path)
print(result_q2_memory)

[('🙏', 7286), ('😂', 3072), ('🚜', 2972), ('🌾', 2363), ('🇮', 2096), ('🇳', 2094), ('🏻', 2080), ('🤣', 1668), ('🏽', 1218), ('👇', 1108)]


Q1_TIME -> Funcion enfocada el optimizar el tiempo

In [32]:
# Función q2_time
def q2_time(file_path: str) -> List[Tuple[str, int]]:
    data = pd.read_json(file_path, lines=True)
    emoji_pattern = re.compile("[\U00010000-\U0010FFFF]", flags=re.UNICODE)
    data['emojis'] = data['content'].apply(lambda x: emoji_pattern.findall(x))
    emoji_counts = data['emojis'].explode().value_counts().head(10)
    return list(emoji_counts.items())


In [33]:
# Probar la función q2_time
result_q2_time = q2_time(file_path)
print(result_q2_time)

[('🙏', 7286), ('😂', 3072), ('🚜', 2972), ('🌾', 2363), ('🇮', 2096), ('🇳', 2094), ('🏻', 2080), ('🤣', 1668), ('🏽', 1218), ('👇', 1108)]


03_MEMORY -> Funcion enfocada en optimizar los recursos

In [37]:
# Función q3_memory
def q3_memory(file_path: str) -> List[Tuple[str, int]]:
    mention_counts = {}
    chunksize = 100000
    for chunk in pd.read_json(file_path, lines=True, chunksize=chunksize):
        mentions = chunk['content'].str.extractall(r'(@\w+)')[0]
        for mention in mentions:
            mention_counts[mention] = mention_counts.get(mention, 0) + 1
    top_mentions = sorted(mention_counts.items(), key=lambda x: x[1], reverse=True)[:10]
    return top_mentions

In [38]:

# Probar la función q3_memory
result_q3_memory = q3_memory(file_path)
print(result_q3_memory)


[('@narendramodi', 2261), ('@Kisanektamorcha', 1836), ('@RakeshTikaitBKU', 1639), ('@PMOIndia', 1422), ('@RahulGandhi', 1125), ('@GretaThunberg', 1046), ('@RaviSinghKA', 1015), ('@rihanna', 972), ('@UNHumanRights', 962), ('@meenaharris', 925)]


Q1_TIME -> Funcion enfocada el optimizar el tiempo

In [39]:
# Función q3_time
def q3_time(file_path: str) -> List[Tuple[str, int]]:
    data = pd.read_json(file_path, lines=True)
    mention_counts = data['content'].str.extractall(r'(@\w+)')[0].value_counts().head(10)
    return list(mention_counts.items())

In [40]:
# Probar la función q3_time
result_q3_time = q3_time(file_path)
print(result_q3_time)

[('@narendramodi', 2261), ('@Kisanektamorcha', 1836), ('@RakeshTikaitBKU', 1639), ('@PMOIndia', 1422), ('@RahulGandhi', 1125), ('@GretaThunberg', 1046), ('@RaviSinghKA', 1015), ('@rihanna', 972), ('@UNHumanRights', 962), ('@meenaharris', 925)]


RESULTADO FINAL Ordenados en tablas

In [46]:
# Resultados de la función q1_memory
result_q1_memory = q1_memory(file_path)
df_q1_memory = pd.DataFrame(result_q1_memory, columns=['Date', 'Top User'])
print("Las top 10 fechas donde hay más tweets (q1_memory):")
display(df_q1_memory)

# Resultados de la función q1_time
result_q1_time = q1_time(file_path)
df_q1_time = pd.DataFrame(result_q1_time, columns=['Username', 'Mentions'])
print("Las top 10 fechas donde hay más tweets (q1_memory):")
display(df_q1_time)

# Resultados de la función q2_memory
result_q2_memory = q2_memory(file_path)
df_q2_memory = pd.DataFrame(result_q2_memory, columns=['Emoji', 'Count'])
print("Los top 10 emojis más usados con su respectivo conteo (q2_memory):")
display(df_q2_memory)

# Resultados de la función q2_time
result_q2_time = q2_time(file_path)
df_q2_time = pd.DataFrame(result_q2_time, columns=['Emoji', 'Count'])
print("Los top 10 emojis más usados con su respectivo conteo (q2_time):")
display(df_q2_time)

# Resultados de la función q3_memory
result_q3_memory = q3_memory(file_path)
df_q3_memory = pd.DataFrame(result_q3_memory, columns=['Username', 'Mentions'])
print("El top 10 histórico de usuarios (username) más influyentes en función del conteo de las menciones (@) que registra cada uno de ellos (q3_memory):")
display(df_q3_memory)

# Resultados de la función q3_time
result_q3_time = q3_time(file_path)
df_q3_time = pd.DataFrame(result_q3_time, columns=['Username', 'Mentions'])
print("El top 10 histórico de usuarios (username) más influyentes en función del conteo de las menciones (@) que registra cada uno de ellos (q3_time):")
display(df_q3_time)


Las top 10 fechas donde hay más tweets (q1_memory):


,Date,Top User
0,2021-02-12,RanbirS00614606
1,2021-02-13,MaanDee08215437
2,2021-02-17,RaaJVinderkaur
3,2021-02-16,jot__b
4,2021-02-14,rebelpacifist
5,2021-02-18,neetuanjle_nitu
6,2021-02-15,jot__b
7,2021-02-20,MangalJ23056160
8,2021-02-23,Surrypuria
9,2021-02-19,Preetm91


usuario (username) que más publicaciones tiene por cada uno de esos días (q1_time):


,Username,Mentions
0,2021-02-12,RanbirS00614606
1,2021-02-13,MaanDee08215437
2,2021-02-17,RaaJVinderkaur
3,2021-02-16,jot__b
4,2021-02-14,rebelpacifist
5,2021-02-18,neetuanjle_nitu
6,2021-02-15,jot__b
7,2021-02-20,MangalJ23056160
8,2021-02-23,Surrypuria
9,2021-02-19,Preetm91


Los top 10 emojis más usados con su respectivo conteo (q2_memory):


...

,Emoji,Count
0,🙏,7286
1,😂,3072
2,🚜,2972
3,🌾,2363
4,🇮,2096
5,🇳,2094
6,🏻,2080
7,🤣,1668
8,🏽,1218
9,👇,1108


El top 10 histórico de usuarios (username) más influyentes en función del conteo de las menciones (@) que registra cada uno de ellos (q3_memory):


,Username,Mentions
0,@narendramodi,2261
1,@Kisanektamorcha,1836
2,@RakeshTikaitBKU,1639
3,@PMOIndia,1422
4,@RahulGandhi,1125
5,@GretaThunberg,1046
6,@RaviSinghKA,1015
7,@rihanna,972
8,@UNHumanRights,962
9,@meenaharris,925


El top 10 histórico de usuarios (username) más influyentes en función del conteo de las menciones (@) que registra cada uno de ellos (q3_time):


,Username,Mentions
0,@narendramodi,2261
1,@Kisanektamorcha,1836
2,@RakeshTikaitBKU,1639
3,@PMOIndia,1422
4,@RahulGandhi,1125
5,@GretaThunberg,1046
6,@RaviSinghKA,1015
7,@rihanna,972
8,@UNHumanRights,962
9,@meenaharris,925


Ahora vamos a comprobar el rendimiento de las formulas enfocadas en memoria y tiempo, para poder concluir con el analisis. 

In [58]:
# Medir rendimiento de q1_time y q1_memory
exec_time_q1_time, mem_usage_q1_time = measure_performance(q1_time, file_path)
exec_time_q1_memory, mem_usage_q1_memory = measure_performance(q1_memory, file_path)

# Medir rendimiento de q2_time y q2_memory
exec_time_q2_time, mem_usage_q2_time = measure_performance(q2_time, file_path)
exec_time_q2_memory, mem_usage_q2_memory = measure_performance(q2_memory, file_path)

# Medir rendimiento de q3_time y q3_memory
exec_time_q3_time, mem_usage_q3_time = measure_performance(q3_time, file_path)
exec_time_q3_memory, mem_usage_q3_memory = measure_performance(q3_memory, file_path)

# Crear DataFrames para mostrar los resultados
results_q1 = {
    "Function": ["q1_time", "q1_memory"],
    "Execution Time (s)": [exec_time_q1_time, exec_time_q1_memory],
    "Max Memory Usage (MiB)": [mem_usage_q1_time, mem_usage_q1_memory]
}

results_q2 = {
    "Function": ["q2_time", "q2_memory"],
    "Execution Time (s)": [exec_time_q2_time, exec_time_q2_memory],
    "Max Memory Usage (MiB)": [mem_usage_q2_time, mem_usage_q2_memory]
}

results_q3 = {
    "Function": ["q3_time", "q3_memory"],
    "Execution Time (s)": [exec_time_q3_time, exec_time_q3_memory],
    "Max Memory Usage (MiB)": [mem_usage_q3_time, mem_usage_q3_memory]
}

df_results_q1 = pd.DataFrame(results_q1).round(2).sort_values(by="Execution Time (s)").reset_index(drop=True)
df_results_q2 = pd.DataFrame(results_q2).round(2).sort_values(by="Execution Time (s)").reset_index(drop=True)
df_results_q3 = pd.DataFrame(results_q3).round(2).sort_values(by="Execution Time (s)").reset_index(drop=True)

# Mostrar resultados
print("Performance Results for Q1:")
display(df_results_q1)

print("Performance Results for Q2:")
display(df_results_q2)

print("Performance Results for Q3:")
display(df_results_q3)

Performance Results for Q1:


,Function,Execution Time (s),Max Memory Usage (MiB)
0,q1_memory,5.61,4504.84
1,q1_time,7.73,4745.98


Performance Results for Q2:


,Function,Execution Time (s),Max Memory Usage (MiB)
0,q2_time,5.57,4975.75
1,q2_memory,6.38,4510.39


Performance Results for Q3:


,Function,Execution Time (s),Max Memory Usage (MiB)
0,q3_time,5.47,4884.39
1,q3_memory,6.23,4593.52


Pros y Contras
Time: 
    Pros: Ejecuta los procesos en menor tiempo ya que el tiempo es su prioridad.
    Contra: Al ser el tiempo su prioridad no se preocupa por los recursos, por lo que podría no completar su ejecución si el entonrno no cumple con los requisitos.  

Memory:
    Pros: Ocupa menos memoria debido al procesamiento en chunks, puede ser efectivo en un entorno sin tantos recursos. 
    Contra: Puede tomar más tiempo de ejecución, debido a que se ejecuta por partes.

Cabe destacar que antes de tomar una decisión sobre cuál solución escoger se deben evaluar diversos aspectos, como por ejemplo la disponibilidad de recursos, la cantidad de RAM y la capacidad de procesamiento del sistema.

Todo depende del objetivo del proyecto.